In [1]:
import os
import uuid

import fire
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger
from termcolor import colored


def main():
    client = LlamaStackClient(
        base_url="http://localhost:8321"
    )

    available_models = [
        model.identifier for model in client.models.list() if model.model_type == "llm"
    ]
    print(available_models)
    
    if not available_models:
        print(colored("No available models. Exiting.", "red"))
        return

    selected_model = available_models[0]
    print(colored(f"Using model: {selected_model}", "green"))

    # Initialize ReActAgent with just websearch
    agent = ReActAgent(
        client=client,
        model=selected_model,
        # model="mistralai/Mistral-7B-Instruct-v0.3",
        # model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
        tools=[
            "mcp::weather",
            ],
        response_format={
            "type": "json_schema",
            "json_schema": ReActOutput.model_json_schema(),
        },
        sampling_params={
            "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
        }
    )

    session_id = agent.create_session(f"react-session-{uuid.uuid4().hex}")

    user_prompts = [
        "Find out what is the best day this week to go for a walk in New York and explain why?",
    ]

    for prompt in user_prompts:
        print(colored(f"User> {prompt}", "blue"))
        response = agent.create_turn(
            messages=[{"role": "user", "content": prompt}],
            session_id=session_id,
            stream=True,
        )

        for log in EventLogger().log(response):
            log.print()

In [2]:
if __name__ == "__main__":
    fire.Fire(main)

['ibm-granite/granite-3.3-2b-instruct', 'meta-llama/Llama-Guard-3-8B']
Using model: ibm-granite/granite-3.3-2b-instruct
User> Find out what is the best day this week to go for a walk in New York and explain why?
inference> {
    "thought": "To answer this question, I need to retrieve the weather forecast for New York this week and identify a day with favorable conditions for a walk. I will use the 'getforecast' tool.",
    "action": {
        "tool_name": "getforecast",
        "tool_params": [
            {
                "name": "latitude",
                "value": "40.7128"
            },
            {
                "name": "longitude",
                "value": "-74.0060"
            }
        ]
    },
    "answer": null
}
tool_execution> Tool:getforecast Response:{"type":"text","text":"Forecast for 40.7128, -74.0060:\n\nOvernight:\nTemperature: 66°F\nWind: 3 mph E\nAreas Of Fog\n---\nTuesday:\nTemperature: 76°F\nWind: 3 to 7 mph SE\nAreas Of Fog\n---\nTuesday Night:\nTemperature

ERROR: Could not consume arg: -f
Usage: ipykernel_launcher.py

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2

/opt/app-root/lib64/python3.11/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
